In [88]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cuda


In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/test.csv')

In [144]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51


In [145]:
train = train.fillna(train['AlogP'].median())

In [110]:
!pip install rdkit-pypi

In [146]:
from rdkit import DataStructs
from rdkit.Chem import PandasTools,AllChem

PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')

def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

# FPs column 추가
train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)

In [147]:
# 사용할 column만 추출
train = train[['FPs', 'MLM', 'HLM']]
test = test[['FPs']]

In [148]:
class CustomDataset(Dataset):
    def __init__(self, df, target, transform, is_test=False):
        self.df = df
        self.target = target # HLM or MLM
        self.is_test = is_test # train,valid / test

        self.feature_select = transform
        if not self.is_test:
            self.fp = self.feature_select.fit_transform(np.stack(df['FPs']))
            # self.fp = np.stack(df['FPs'])
        else: # valid or test
            self.fp = self.feature_select.transform(np.stack(df['FPs']))
            #self.fp = np.stack(df['FPs'])

    def __getitem__(self, index):
        fp = self.fp[index]
        if not self.is_test: # test가 아닌 경우(label 존재)
            label = self.df[self.target][index]
            return torch.tensor(fp).to(device).float(), torch.tensor(label).to(device).float().unsqueeze(dim=-1) # feature, label

        else: # test인 경우
            return torch.tensor(fp).to(device).float() # feature

    def __len__(self):
        return len(self.df)

In [149]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(df=train, target='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(df=train, target='HLM', transform=transform, is_test=False)

input_size = train_MLM.fp.shape[1]
input_size

251

In [150]:
train_HLM.fp.shape

(3498, 251)

In [151]:
torch.tensor(train_MLM.fp[1]).shape, torch.tensor(train['MLM'][1]).float().unsqueeze(dim=-1).shape

(torch.Size([251]), torch.Size([1]))

In [152]:
# Hyperparameter
CFG = {'BATCH_SIZE': 256,
       'EPOCHS': 1000,
       'HIDDEN_SIZE': 180,
       'OUTPUT_SIZE': 1,
       'DROPOUT_RATE': 0,
       'FEATURE_SIZE': 1,
       'NUM_LAYERS' : 4,
       'LEARNING_RATE': 1e-5
      }

In [153]:
# train,valid split
train_MLM_dataset, valid_MLM_dataset = train_test_split(train_MLM, test_size=0.2, random_state=42)
train_HLM_dataset, valid_HLM_dataset = train_test_split(train_HLM, test_size=0.2, random_state=42)

In [154]:
train_MLM_loader = DataLoader(dataset=train_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_MLM_loader = DataLoader(dataset=valid_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)


train_HLM_loader = DataLoader(dataset=train_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_HLM_loader = DataLoader(dataset=valid_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)

In [155]:
X_train, y_train = next(iter(train_MLM_loader))
print (X_train.shape, y_train.shape)

torch.Size([256, 251]) torch.Size([256, 1])


### 모델 생성

In [156]:
class Net(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, dropout_p, output_size):
        super(Net, self).__init__()

        self.sequenceclassifier = nn.LSTM(
                input_size = feature_size,
                hidden_size = hidden_size,
                num_layers = num_layers,
                batch_first = True,
                dropout = dropout_p,
                bidirectional = True
        )
        # (256, 251, 1) -> (256, 251, 96)

        self.fc = nn.Sequential(
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(hidden_size * 2),
            nn.Linear(hidden_size * 2, output_size)
        )
      # (256, 96) -> (256, 1)

    def forward(self, x):
        # |x| = (256, 251)
        output, _ = self.sequenceclassifier(x) # |output| = (256, 251, 96)
        output = output[:, -1, :] # |output| = (256, 96)
        y = self.fc(output)
        return y

In [157]:
model_MLM = Net(CFG['FEATURE_SIZE'],CFG['HIDDEN_SIZE'],CFG['NUM_LAYERS'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE']).to(device)
model_HLM = Net(CFG['FEATURE_SIZE'],CFG['HIDDEN_SIZE'],CFG['NUM_LAYERS'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE']).to(device)

In [158]:
import torch.nn as nn

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()  # 기존의 MSELoss 함수 사용

    def forward(self, output, target):
        mse_loss = self.mse(output, target)  # 기존의 MSELoss를 계산
        rmse_loss = torch.sqrt(mse_loss)  # MSE에 제곱근 씌워 RMSE 계산
        return rmse_loss

criterion = RMSELoss()
optimizer_MLM = torch.optim.Adam(model_MLM.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_HLM = torch.optim.Adam(model_HLM.parameters(), lr=CFG['LEARNING_RATE'])


### training 함수 선언

In [159]:
def train(train_loader, valid_loader, model, criterion, optimizer, epochs):
    model.train()

    for epoch in range(epochs):
        running_loss = 0
        for inputs, targets in train_loader:

            num_samples, num_features = inputs.shape

            # LSTM 입력 형태에 맞게 변경
            sequence_length = num_features  # 각 벡터를 시퀀스로 취급
            input_features = 1  # 각 값이 이진 값이므로 하나의 특성만 있음

            inputs_reshaped = inputs.view(num_samples, sequence_length, input_features).to(device)

            # |x| = (256, 251) or (238, 251)
            optimizer.zero_grad()

            output = model(inputs_reshaped)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if epoch % 100 == 0:
            valid_loss = 0
            with torch.no_grad():
                for inputs, targets in valid_loader:
                    num_samples, num_features = inputs.shape

                    val_inputs_reshaped = inputs.view(num_samples, sequence_length, input_features).to(device)

                    output = model(val_inputs_reshaped)
                    loss = criterion(output, targets)
                    valid_loss += loss.item()

            print(f'Epoch: {epoch}/{epochs}, Train Loss: {running_loss/len(train_loader)}, Valid Loss: {valid_loss/len(valid_HLM_loader)}')

            model.train()

    return model

In [160]:
print("Training Start: MLM")
model_MLM = train(train_MLM_loader, valid_MLM_loader, model_MLM, criterion, optimizer_MLM, epochs=CFG['EPOCHS'])

print("Training Start: HLM")
model_HLM = train(train_HLM_loader, valid_HLM_loader, model_HLM, criterion, optimizer_HLM, epochs=CFG['EPOCHS'])

Training Start: MLM
Epoch: 0/1000, Train Loss: 51.41046420010653, Valid Loss: 52.48161824544271


KeyboardInterrupt: ignored

### Save Model

In [47]:
torch.save(model_MLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/23.08.12_model_MLM.pth')  # 모델 객체의 state_dict 저장
torch.save(model_HLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/23.08.12_model_HLM.pth')

### Load Model

In [ ]:
model_MLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/23.08.12_model_MLM.pth'))
model_HLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/23.08.12_model_HLM.pth'))

In [48]:
test_MLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)
test_HLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)

test_MLM_loader = DataLoader(dataset=test_MLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

test_HLM_loader = DataLoader(dataset=test_HLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

In [49]:
def inference(test_loader, model):
    model.eval()
    preds = []

    with torch.no_grad():
        for inputs in test_loader:

            num_samples, num_features = inputs.shape

            # LSTM 입력 형태에 맞게 변경
            sequence_length = num_features  # 각 벡터를 시퀀스로 취급
            input_features = 1  # 각 값이 이진 값이므로 하나의 특성만 있음

            inputs_reshaped = inputs.view(num_samples, sequence_length, input_features).to(device)

            output = model(inputs_reshaped)
            preds.extend(output.cpu().numpy().flatten().tolist())

    return preds

In [50]:
predictions_MLM = inference(test_MLM_loader, model_MLM)
predictions_HLM = inference(test_HLM_loader, model_HLM)

In [51]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/sample_submission.csv')

submission['MLM'] = predictions_MLM
submission['HLM'] = predictions_HLM
submission

,id,MLM,HLM
0,TEST_000,11.407097,9.432817
1,TEST_001,6.846638,5.977738
2,TEST_002,-4.805486,-0.986147
3,TEST_003,-4.750093,-5.041346
4,TEST_004,6.732722,13.458108
...,...,...,...
478,TEST_478,4.018270,11.241875
479,TEST_479,12.142907,25.131264
480,TEST_480,-5.160524,0.771250
481,TEST_481,11.413135,11.973424


In [52]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/submissions/23.08.12_submission.csv', index=False)